In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical


from tensorflow.keras.applications import VGG16

# Завантаження VGG16 моделі
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False


In [2]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Масштабуємо зображення до діапазону [0, 1]
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# Кодуємо мітки у формат one-hot
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


In [3]:
import numpy as np
from tensorflow.image import resize

# Збільшення розміру зображень
train_images_resized = resize(train_images, [32, 32])
test_images_resized = resize(test_images, [32, 32])

# Конвертація до RGB
train_images_rgb = np.repeat(train_images_resized, 3, -1)
test_images_rgb = np.repeat(test_images_resized, 3, -1)


In [4]:
model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)),  # Corrected input shape
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),
    metrics=["acc"]
)
base_model.trainable = False

history = model.fit(train_images_rgb, train_labels, epochs=20, batch_size=64, validation_data=(test_images_rgb, test_labels))


Epoch 1/20
938/938 [==============================] - 78s 82ms/step - loss: 0.3283 - acc: 0.3934 - val_loss: 0.1932 - val_acc: 0.6242
Epoch 2/20
938/938 [==============================] - 78s 83ms/step - loss: 0.1678 - acc: 0.6823 - val_loss: 0.1534 - val_acc: 0.6973
Epoch 3/20
938/938 [==============================] - 76s 81ms/step - loss: 0.1407 - acc: 0.7193 - val_loss: 0.1354 - val_acc: 0.7233
Epoch 4/20
938/938 [==============================] - 74s 79ms/step - loss: 0.1272 - acc: 0.7404 - val_loss: 0.1269 - val_acc: 0.7366
Epoch 5/20
938/938 [==============================] - 76s 81ms/step - loss: 0.1187 - acc: 0.7592 - val_loss: 0.1197 - val_acc: 0.7575
Epoch 6/20
938/938 [==============================] - 86s 92ms/step - loss: 0.1128 - acc: 0.7724 - val_loss: 0.1153 - val_acc: 0.7623
Epoch 7/20
938/938 [==============================] - 87s 93ms/step - loss: 0.1081 - acc: 0.7815 - val_loss: 0.1091 - val_acc: 0.7783
Epoch 8/20
938/938 [==============================] - 87s 92ms

In [5]:
test_loss, test_acc = model.evaluate(test_images_rgb, test_labels)
print(f"Test accuracy: {test_acc}")


313/313 [==============================] - 4s 13ms/step - loss: 0.0867 - acc: 0.8298
Test accuracy: 0.829800009727478


In [6]:
from sklearn.metrics import classification_report

# Отримання передбачень
predicted_probs = model.predict(test_images_rgb)
predicted_labels = np.argmax(predicted_probs, axis=1)

# Перетворення реальних міток
true_labels = np.argmax(test_labels, axis=1)

# Отримання звіту про класифікацію
report = classification_report(true_labels, predicted_labels)
print(report)


313/313 [==============================] - 4s 13ms/step
              precision    recall  f1-score   support

           0       0.81      0.79      0.80      1000
           1       0.99      0.96      0.97      1000
           2       0.64      0.79      0.71      1000
           3       0.81      0.88      0.84      1000
           4       0.67      0.76      0.71      1000
           5       0.96      0.93      0.94      1000
           6       0.63      0.39      0.48      1000
           7       0.91      0.92      0.92      1000
           8       0.94      0.94      0.94      1000
           9       0.93      0.96      0.94      1000

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000

